In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np

# Enable mixed precision for faster training
tf.keras.mixed_precision.set_global_policy('mixed_float16')


The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [ ]:

# Load and preprocess Fashion MNIST data
from tensorflow.keras.datasets import fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Reshape images and normalize
train_images = np.expand_dims(train_images, -1)
test_images = np.expand_dims(test_images, -1)

# Resize images to 96x96 and normalize
train_images = tf.image.resize(train_images, (96, 96)) / 255.0
test_images = tf.image.resize(test_images, (96, 96)) / 255.0


4422102/4422102 [==============================] - 0s 0us/step


In [ ]:

num_classes = 10

# Load the pretrained MobileNetV3Small model
base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(96, 96, 3))
base_model.trainable = False  # Freeze all layers



4334752/4334752 [==============================] - 0s 0us/step


In [ ]:

inputs = tf.keras.Input(shape=(96, 96, 1))
x = Conv2D(3, (3, 3), padding='same')(inputs)  # Convert grayscale to RGB


In [ ]:

# Pass through the pretrained model
x = base_model(x, training=False)

# Add global pooling and dense layers
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax', dtype='float32')(x)


In [ ]:

# Build the model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:

# Set up data augmentation with a smaller batch size
train_datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, horizontal_flip=True)
train_generator = train_datagen.flow(train_images, train_labels, batch_size=32)


In [ ]:

# Early stopping and model checkpoint to save the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, mode='max')

# Train for a smaller number of epochs
history = model.fit(train_generator, epochs=5,
                    validation_data=(test_images, test_labels),
                    callbacks=[early_stopping, checkpoint])

#

Epoch 1/5
1875/1875 [==============================] - 961s 511ms/step - loss: 1.0055 - accuracy: 0.6250 - val_loss: 0.6463 - val_accuracy: 0.7530


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/5
1875/1875 [==============================] - 954s 509ms/step - loss: 0.6359 - accuracy: 0.7647 - val_loss: 0.5198 - val_accuracy: 0.8081
Epoch 3/5
1875/1875 [==============================] - 955s 510ms/step - loss: 0.5652 - accuracy: 0.7901 - val_loss: 0.4768 - val_accuracy: 0.8214
Epoch 4/5
1875/1875 [==============================] - 955s 509ms/step - loss: 0.5281 - accuracy: 0.8045 - val_loss: 0.4803 - val_accuracy: 0.8182
Epoch 5/5
1875/1875 [==============================] - 954s 509ms/step - loss: 0.5029 - accuracy: 0.8131 - val_loss: 0.4664 - val_accuracy: 0.8265


In [ ]:

model.load_weights("best_model.h5")
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


313/313 [==============================] - 36s 114ms/step - loss: 0.4664 - accuracy: 0.8265
Test accuracy: 82.65%


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the saved model
model = tf.keras.models.load_model("best_model.h5")

# Function to preprocess and predict on a new image
def preprocess_and_predict(img_path):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=(96, 96), color_mode='grayscale')  # Load as grayscale
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array = tf.image.resize(img_array, (96, 96))  # Resize if needed
    # The line below was causing the error, as the model expected grayscale input,
    #  but it was receiving an RGB image due to this conversion
    # img_array = tf.image.grayscale_to_rgb(img_array)  # Remove this line
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1]

    # Make a prediction
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=-1)

    return predicted_class[0]

# Example usage: Replace 'path/to/your/image.jpg' with the path to your image
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
img_path = '/content/trouser.jpg'
predicted_class = preprocess_and_predict(img_path)
print(f"Predicted class: {predicted_class}")
print("Type of product image uploaded is ",class_names[predicted_class])


1/1 [==============================] - 1s 854ms/step
Predicted class: 1
Type of product image uploaded is  Trouser


In [ ]:

img_path = '/content/shirt.jpeg'
predicted_class = preprocess_and_predict(img_path)
print(f"Predicted class: {predicted_class}")
print("Type of product image uploaded is ",class_names[predicted_class])

1/1 [==============================] - 0s 43ms/step
Predicted class: 0
Type of product image uploaded is  T-shirt/top
